<a href="https://colab.research.google.com/github/NeuroWinter/news-headline-ai/blob/master/Spacy_transfomers_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install -U spacy[cuda100]
!pip install spacy-transformers
!python3 -m spacy download en_trf_distilbertbaseuncased_lg
!pip install torch

Requirement already up-to-date: spacy[cuda100] in /usr/local/lib/python3.6/dist-packages (2.2.3)
     |████████████████████████████████| 382.9MB 42kB/s 


✔ Download and installation successful
You can now load the model via spacy.load('en_trf_distilbertbaseuncased_lg')


In [1]:
import pandas as pd
import csv

# REMEBER TO UPLOAD THIS!
input_file = 'news_headlines.csv'

TRAIN_DATA = []
df = pd.read_csv(input_file)

list_of_sites = list(df['news_site'].unique())

for row in df.iterrows():
    row_headline       = row[1][2]
    row_cat            = {site: 0.0 for site in list_of_sites}
    row_cat[row[1][1]] += 1.0
    TRAIN_DATA.append((row_headline.strip(), {"cats": row_cat}))
print(TRAIN_DATA[0])
df = None

('Experts say adapting to climate change can pay off manifold', {'cats': {'ABC - Tech': 1.0, 'ABC - Top Stories': 0.0, 'ABC - US': 0.0, 'ABC - World': 0.0, 'Al Jazeera': 0.0, 'AllAfrica News - Mozambique': 0.0, 'ARRL': 0.0, 'Ars Technica - all features': 0.0, 'Ars Technica - all news': 0.0, 'Axios - Business': 0.0, 'Axios - Energy': 0.0, 'Axios - Future': 0.0, 'Axios - Health Care': 0.0, 'Axios - Politics': 0.0, 'Axios - Science': 0.0, 'Axios - Technology': 0.0, 'Axios - Top': 0.0, 'Axios - World': 0.0, 'Bandladesh - bdnews24.com': 0.0, 'Bangkok Post - Lifestyle': 0.0, 'Bangkok Post - Most Recent': 0.0, 'Bangkok Post - Top Stories': 0.0, 'Bangkok Post - Travel': 0.0, 'BBC': 0.0, 'BBC - Africa': 0.0, 'BBC - Asia': 0.0, 'BBC - England': 0.0, 'BBC - Europe': 0.0, 'BBC - Latin America': 0.0, 'BBC - Middle East': 0.0, 'BBC - Northern Ireland': 0.0, 'BBC - Scotland': 0.0, 'BBC - UK': 0.0, 'BBC - US & Canada': 0.0, 'BBC - Wales': 0.0, 'BBC - World': 0.0, 'Breitbart "News"': 0.0, 'Buzzfeed - U

In [0]:
# RUN THIS TO MAKE SURE THAT SPACY IS WORKING
import spacy

nlp = spacy.load('en_trf_distilbertbaseuncased_lg')
doc = nlp("Apple shares rose on the news. Apple pie is delicious.")
print(doc[0].similarity(doc[7]))
print(doc._.trf_last_hidden_state.shape)

0.7756554
(16, 768)


In [0]:
TRAIN_DATA = [
    ("text1", {"cats": {"POSITIVE": 1.0, "NEGATIVE": 0.0, "TESTING": 0.0}})
]

In [0]:
import spacy
import random
import torch

from spacy.util         import minibatch
from spacy_transformers import TransformersLanguage, TransformersWordPiecer, TransformersTok2Vec

BATCH_SIZE = 1000
ITERATIONS = 2
MODEL      = "distilbert-base-uncased"

is_using_gpu = spacy.prefer_gpu()
if is_using_gpu:
    print("USING GPU")
    torch.set_default_tensor_type("torch.cuda.FloatTensor")

# nlp = TransformersLanguage(trf_name=MODEL, meta={"lang": "en"})
# nlp.add_pipe(nlp.create_pipe("sentencizer"))
# nlp.add_pipe(TransformersWordPiecer.from_pretrained(nlp.vocab, MODEL))
# nlp.add_pipe(TransformersTok2Vec.from_pretrained(nlp.vocab, MODEL))
# print(nlp.pipe_names)  # ['sentencizer', 'trf_wordpiecer', 'trf_tok2vec']

nlp = spacy.load("en_trf_distilbertbaseuncased_lg")
print(nlp.pipe_names) # ["sentencizer", "trf_wordpiecer", "trf_tok2vec"]
textnews = nlp.create_pipe("trf_textcat", config={"exclusive_classes": True})
# for label in ("POSITIVE", "NEGATIVE", "TESTING"):
for label in list_of_sites:
    textnews.add_label(label)
nlp.add_pipe(textnews)
optimizer = nlp.resume_training()
for i in range(ITERATIONS):
    random.shuffle(TRAIN_DATA)
    losses = {}
    for x, batch in enumerate(minibatch(TRAIN_DATA, size=BATCH_SIZE)):
        print(x)
        texts, news = zip(*batch)
        nlp.update(texts, news, sgd=optimizer, losses=losses)
        print(losses)
    print(i, losses)

USING GPU
['sentencizer', 'trf_wordpiecer', 'trf_tok2vec']
0
{'trf_textcat': 9.977775334846228e-07}
1
{'trf_textcat': 1.994826789086801e-06}
2
{'trf_textcat': 2.9910064540672465e-06}
3
{'trf_textcat': 3.992309643763292e-06}
4
{'trf_textcat': 4.992492790734104e-06}
5
{'trf_textcat': 5.9879840819121455e-06}
6
{'trf_textcat': 6.980743933127087e-06}
7
{'trf_textcat': 7.972201728989603e-06}
8
{'trf_textcat': 8.962144306678965e-06}
9
{'trf_textcat': 9.951846323019709e-06}
10
{'trf_textcat': 1.0940934544123593e-05}
11
{'trf_textcat': 1.1929917604902585e-05}
12
{'trf_textcat': 1.2917545859636448e-05}
13
{'trf_textcat': 1.3904702541367442e-05}
14
{'trf_textcat': 1.4894941273269069e-05}
15
{'trf_textcat': 1.5882872844485973e-05}
16
{'trf_textcat': 1.687327335275768e-05}
17
{'trf_textcat': 1.786133134373813e-05}
18
{'trf_textcat': 1.88504571951853e-05}
19
{'trf_textcat': 1.98383040697081e-05}
20
{'trf_textcat': 2.0827467778872233e-05}
21
{'trf_textcat': 2.1816695834786515e-05}
22
{'trf_textcat': 

In [8]:
a = nlp("RANDOM TEXT")
a.cats

{'NEGATIVE': 0.03571971505880356,
 'POSITIVE': 0.9285604953765869,
 'TESTING': 0.03571971505880356}

In [3]:
from spacy_transformers import TransformersLanguage, TransformersWordPiecer, TransformersTok2Vec

name = "bert-base-uncased"
nlp = TransformersLanguage(trf_name=name, meta={"lang": "en"})
nlp.add_pipe(nlp.create_pipe("sentencizer"))
nlp.add_pipe(TransformersWordPiecer.from_pretrained(nlp.vocab, name))
nlp.add_pipe(TransformersTok2Vec.from_pretrained(nlp.vocab, name))
print(nlp.pipe_names)  # ['sentencizer', 'trf_wordpiecer', 'trf_tok2vec']

100%|██████████| 440473133/440473133 [00:07<00:00, 57110878.94B/s]


['sentencizer', 'trf_wordpiecer', 'trf_tok2vec']


In [0]:
textnews = nlp.create_pipe("trf_textcat", config={"exclusive_classes": True})
for label in list_of_sites:
    textnews.add_label(label)
nlp.add_pipe(textnews)